# Blood Pressure

In [ ]:
def bp(start, end, patient_id, print1, print2):
    if patient_id == 'simulator':
        time = pickle.load(open('samples/simulated_time', 'rb'))
        bp = pickle.load(open('samples/simulated_processed_bp', 'rb'))
        return time[(int)(start*F):(int)(end*F)], bp[(int)(start*F):(int)(end*F)]
    
    debug = print1 or print2
    if debug == False and os.path.exists('pickles/'+str(patient_id)+'-bp-processed') == True:
        time = pickle.load(open('pickles/'+str(patient_id)+'-time-processed', 'rb'))
        bp = pickle.load(open('pickles/'+str(patient_id)+'-bp-processed', 'rb'))
        return time[(int)(start*F):(int)(end*F)], bp[(int)(start*F):(int)(end*F)]
    
    if debug == False:
        start_ = 0
        end_ = 2300
    else:
        start_ = start
        if start != 0:
            start_ = start - MARGIN
        end_ = end + MARGIN
    bp = []
    filename = patient_id + "-MDC_PRESS_BLD_ART_ABP-125.csv"
    if print1 or print2:
        print(filename+':')
        print("sampling frequency is ", F," Hz")
    with open('samples/'+ filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            bp.append((0.0625*float(row[1]))-40)

    bp_small = np.array(bp[start_*125:end_*125])
    peaks,_= find_peaks(bp_small, prominence=1, distance=42)
    peaks_sec = (peaks + start_*125) * 0.008
    sys = bp_small[peaks]
    time = np.arange(start_, end_, 0.008)

    if print1:
        plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
        plt.plot(time,bp_small)
        plt.plot(peaks_sec, sys, "x")
        plt.title('Blood Pressure')
        plt.xlabel('Time [sec]')
        plt.show()

    sys_interp_func = interp1d(peaks_sec, sys)
    time = np.arange(start_ + MARGIN, end_ - MARGIN + 1/F, 1/F)

    if print2:
        plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
        plt.plot(peaks_sec,sys) 
        plt.plot(time,sys_interp_func(time))
        plt.title('Systolic Blood Pressure')
        plt.xlabel('Time [sec]')
        plt.legend(['data', 'resampled'], loc='best')
        plt.show()
    
    if debug == False:
        with open('pickles/'+str(patient_id)+'-time-processed', 'wb') as time_file:
            pickle.dump(time, time_file)
        with open('pickles/'+str(patient_id)+'-bp-processed', 'wb') as bp_file:
            pickle.dump(sys_interp_func(time), bp_file)
        return time[(int)(start*F):(int)(end*F)], bp[(int)(start*F):(int)(end*F)]
